In [2]:
%pip install -q wandb
import wandb
wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [1]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
from Diffusion import DiffusionModel
from utils import *
import wandb

In [9]:
img_size = 32
batch_size = 32
epochs = 100
learning_rate = 0.0001
log = False
noise_steps = 1000

In [10]:
transform = transforms.Compose([
            transforms.Resize(32),
            #transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # normalize mean and std for n=3 color channels
        ])

In [11]:
training_set = torchvision.datasets.CIFAR10('./data', train=True, transform=transform, download=True)
testing_set = torchvision.datasets.CIFAR10('./data', train=False, transform=transform, download=True)
training_set, validation_set = torch.utils.data.random_split(training_set, [45000, 5000])

Files already downloaded and verified
Files already downloaded and verified


In [12]:
training_loader = torch.utils.data.DataLoader(training_set, batch_size=batch_size, shuffle=True, num_workers=10,  pin_memory=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size, shuffle=True, num_workers=10, pin_memory=True)
testing_loader = torch.utils.data.DataLoader(testing_set, batch_size=batch_size, shuffle=True, num_workers=10, pin_memory=True)

In [13]:
model = DiffusionModel(
    learning_rate=learning_rate,
    in_channels=3,
    out_channels=3,
    noise_steps=noise_steps,
    beta_start=1e-4,
    beta_end=0.02,
    img_size=img_size,
    device="cuda"
)

In [14]:
path = '/content/drive/MyDrive/Colab Notebooks/model_save.pt'

In [15]:
if log:
    wandb.init(
        project="DenoiseDiffusion",
        config={
        "dataset": "CIFAR-100",
        "img_size": img_size,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "epochs": epochs,
        "Type": "DDPM_V2",
        "Save": "Save-0"
        }
    )

model.train_model(training_loader, validation_loader, epochs, log=log, save_path=path)
wandb.finish()

Epoch 0...


  0%|▍                                                                              | 7/1407 [00:46<2:34:47,  6.63s/it]


KeyboardInterrupt: 